In [2]:
pip install jupyterlab qiskit qiskit-aqua networkx


  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'error'


  error: subprocess-exited-with-error
  
  python setup.py egg_info did not run successfully.
  exit code: 1
  
  [1 lines of output]
  ERROR: Can not execute `setup.py` since setuptools is not available in the build environment.
  [end of output]
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
error: metadata-generation-failed

Encountered error while generating package metadata.

See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


In [ ]:
import networkx as nx
import numpy as np
from qiskit import Aer
from qiskit.aqua import QuantumInstance
from qiskit.aqua.algorithms import QAOA
from qiskit.aqua.operators import WeightedPauliOperator
from qiskit.aqua.input import EnergyInput
from qiskit.aqua.components.optimizers import COBYLA

# Step 1: Build the graph (e.g., graph of IonQ employees with edges representing interactions)
def create_graph():
    # Create a simple undirected graph
    G = nx.Graph()
    
    # Add nodes (employees)
    G.add_nodes_from([1, 2, 3, 4, 5, 6, 7])
    
    # Add edges (friendships/interactions)
    G.add_edges_from([(1, 2), (1, 3), (2, 4), (3, 4), (4, 5), (5, 6), (6, 7)])
    
    return G

# Step 2: Create the problem Hamiltonian for the graph partitioning problem
def create_hamiltonian(G):
    num_nodes = len(G.nodes)
    
    # Initialize the cost matrix for graph partitioning (maximize cross-group edges)
    cost_matrix = np.zeros((num_nodes, num_nodes))
    
    for u, v in G.edges():
        cost_matrix[u-1][v-1] = 1  # Assign a weight to edges (cross-group edges should be maximized)
    
    # Construct the QUBO matrix (Binary Quadratic Unconstrained Optimization)
    # This matrix defines the objective of maximizing the edge cut
    qubo_matrix = np.zeros((num_nodes, num_nodes))
    
    for i in range(num_nodes):
        for j in range(num_nodes):
            qubo_matrix[i][j] = cost_matrix[i][j]
    
    # Construct the operator using Pauli matrices
    pauli_list = []
    for i in range(num_nodes):
        for j in range(i+1, num_nodes):
            weight = qubo_matrix[i][j]
            if weight != 0:
                pauli_list.append([weight, [i, j]])

    # Return the Hamiltonian operator for QAOA
    hamiltonian = WeightedPauliOperator(paulis=pauli_list)
    return hamiltonian

# Step 3: Setup QAOA for optimization
def run_qaoa(hamiltonian):
    # Setup quantum backend
    backend = Aer.get_backend('statevector_simulator')
    
    # Use COBYLA optimizer
    optimizer = COBYLA(maxiter=100)
    
    # Setup the QAOA algorithm
    qaoa = QAOA(hamiltonian, optimizer, p=1)  # p=1 corresponds to one layer of QAOA
    
    # Setup quantum instance
    quantum_instance = QuantumInstance(backend)
    
    # Run the QAOA algorithm
    result = qaoa.run(quantum_instance)
    
    return result

# Step 4: Analyze the result to ensure balanced sizes and connectedness, and calculate scores
def analyze_result(result, G):
    # Decode the result (extract the binary solution)
    solution = result['eigvecs'][0]  # Get the first eigenvector
    print("Solution (binary representation of partition):", solution)
    
    # Convert the binary solution to groups
    group1 = [i+1 for i in range(len(solution)) if solution[i] == 0]
    group2 = [i+1 for i in range(len(solution)) if solution[i] == 1]
    
    print("Group 1:", group1)
    print("Group 2:", group2)
    
    # Check for roughly equal group sizes
    size_diff = abs(len(group1) - len(group2))
    print("Size Difference:", size_diff)
    
    # Check for connectedness of each group
    subgraph1 = G.subgraph(group1)
    subgraph2 = G.subgraph(group2)
    
    connected1 = nx.is_connected(subgraph1)
    connected2 = nx.is_connected(subgraph2)
    
    print(f"Group 1 connected: {connected1}")
    print(f"Group 2 connected: {connected2}")
    
    # Calculate the cut size (number of edges between group1 and group2)
    cut_size = 0
    for u, v in G.edges():
        if (u in group1 and v in group2) or (u in group2 and v in group1):
            cut_size += 1
    
    # Calculate the total number of edges in the graph
    total_edges = len(G.edges())
    print(f"Total edges in the graph: {total_edges}")
    print(f"Cut size (number of edges between groups): {cut_size}")
    
    # Calculate the "score" based on cut size and size balance
    balanced_score = 1 - (size_diff / len(G.nodes))  # Normalize the balance score (0 to 1)
    cut_score = cut_size / total_edges  # Proportion of cut edges in total edges
    
    print(f"Balanced Score: {balanced_score}")
    print(f"Cut Score: {cut_score}")
    
    return group1, group2, connected1, connected2, size_diff, cut_size, balanced_score, cut_score

# Step 5: Execute the solution
def main():
    G = create_graph()  # Create the graph
    hamiltonian = create_hamiltonian(G)  # Create the Hamiltonian
    result = run_qaoa(hamiltonian)  # Run QAOA
    
    # Analyze the results
    group1, group2, connected1, connected2, size_diff, cut_size, balanced_score, cut_score = analyze_result(result, G)

    if connected1 and connected2:
        print("Valid partition found!")
    else:
        print("Invalid partition: One or both groups are disconnected.")

    # Output the scores
    print(f"Score Summary:")
    print(f"Balanced Partition Score: {balanced_score:.4f}")
    print(f"Cut Size Score (cut proportion): {cut_score:.4f}")
    print(f"Total Cut Size: {cut_size}")
    print(f"Size Difference between groups: {size_diff}")

# Run the main function
main()


In [5]:
import networkx as nx
import numpy as np
from qiskit import Aer
from qiskit.aqua import QuantumInstance
from qiskit.aqua.algorithms import QAOA
from qiskit.aqua.operators import WeightedPauliOperator
from qiskit.aqua.input import EnergyInput
from qiskit.aqua.components.optimizers import COBYLA

# Build the graph 
def create_graph():
    # Simple undirected graph
    G = nx.Graph()
    
    # Node
    G.add_nodes_from([1, 2, 3, 4, 5, 6, 7])
    
    # Edges
    G.add_edges_from([(1, 2), (1, 3), (2, 4), (3, 4), (4, 5), (5, 6), (6, 7)])
    
    return G

# Problem Hamiltonian for the graph partitioning problem
def create_hamiltonian(G):
    num_nodes = len(G.nodes)
    
    # Initialize the cost matrix for graph partitioning
    cost_matrix = np.zeros((num_nodes, num_nodes))
    
    for u, v in G.edges():
        cost_matrix[u-1][v-1] = 1  # Assign a weight to edges (cross-group edges should be maximized)
    
    # Construct the QUBO matrix (Binary Quadratic Unconstrained Optimization)
    qubo_matrix = np.zeros((num_nodes, num_nodes))
    
    for i in range(num_nodes):
        for j in range(num_nodes):
            qubo_matrix[i][j] = cost_matrix[i][j]
    
    # Construct operator using Pauli matrices
    pauli_list = []
    for i in range(num_nodes):
        for j in range(i+1, num_nodes):
            weight = qubo_matrix[i][j]
            if weight != 0:
                pauli_list.append([weight, [i, j]])

    # Return Hamiltonian operator for QAOA
    hamiltonian = WeightedPauliOperator(paulis=pauli_list)
    return hamiltonian

# Step 3: Setup QAOA for optimization
def run_qaoa(hamiltonian):
    # Quantum backend
    backend = Aer.get_backend('statevector_simulator')
    
    # COBYLA optimizer
    optimizer = COBYLA(maxiter=100)
    
    # QAOA algorithm
    qaoa = QAOA(hamiltonian, optimizer, p=1)  # p=1 corresponds to one layer of QAOA
    
    # Quantum instance
    quantum_instance = QuantumInstance(backend)
    
    # Run the QAOA algorithm
    result = qaoa.run(quantum_instance)
    
    return result

#  Analyze result 
def analyze_result(result, G):
    # extract binary solution
    solution = result['eigvecs'][0]  # Get the first eigenvector
    print("Solution (binary representation of partition):", solution)
    
    # Convert binary solution to groups
    group1 = [i+1 for i in range(len(solution)) if solution[i] == 0]
    group2 = [i+1 for i in range(len(solution)) if solution[i] == 1]
    
    print("Group 1:", group1)
    print("Group 2:", group2)
    
    # Check roughly equal group sizes
    size_diff = abs(len(group1) - len(group2))
    print("Size Difference:", size_diff)
    
    # Check connectedness
    subgraph1 = G.subgraph(group1)
    subgraph2 = G.subgraph(group2)
    
    connected1 = nx.is_connected(subgraph1)
    connected2 = nx.is_connected(subgraph2)
    
    print(f"Group 1 connected: {connected1}")
    print(f"Group 2 connected: {connected2}")
    
    return group1, group2, connected1, connected2

# Main function
def main():
    G = create_graph()  # Create the graph
    hamiltonian = create_hamiltonian(G)  # Create the Hamiltonian
    result = run_qaoa(hamiltonian)  # Run QAOA
    
    # Analyze the results
    group1, group2, connected1, connected2 = analyze_result(result, G)

    if connected1 and connected2:
        print("Valid partition found!")
    else:
        print("Invalid partition: One or both groups are disconnected.")

# Run the main function
main()


ModuleNotFoundError: No module named 'qiskit'